In [ ]:
import pandas as pd
import nltk
import re

load model

In [ ]:
data=pd.read_csv('/content/drive/MyDrive/IBM/spam.csv',encoding='ISO-8859-1')

In [ ]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
data['v1']=le.fit_transform(data['v1'])

In [ ]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,0,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,0,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,0,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,0,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [ ]:
pe=PorterStemmer()
arr=[]

In [ ]:
data.shape

(5572, 5)

In [ ]:
for i in range(5572):
  msg=data['v2'][i]
  msg=re.sub('[^a-zA-Z]',' ',msg)
  msg=msg.lower()
  msg=msg.split()
  msg=[pe.stem(word) for word in msg if not word in set(stopwords.words('english'))]
  msg=' '.join(msg)
  arr.append(msg)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=8000)

In [ ]:
x=cv.fit_transform(arr).toarray()

In [ ]:
x

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
y=data['v1'].values

In [ ]:
y

array([0, 0, 1, ..., 0, 0, 0])

In [ ]:
x.shape

(5572, 6221)

Train and Test Split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

model building

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model=Sequential()

In [ ]:
model.add(Dense(units=6221,activation='relu'))
model.add(Dense(units=8000,activation='relu'))
model.add(Dense(units=1,activation='sigmoid'))

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x_train,y_train,epochs=5)

Epoch 1/5
140/140 [==============================] - 87s 603ms/step - loss: 0.1193 - accuracy: 0.9684
Epoch 2/5
140/140 [==============================] - 84s 598ms/step - loss: 0.0062 - accuracy: 0.9984
Epoch 3/5
140/140 [==============================] - 85s 607ms/step - loss: 0.0011 - accuracy: 0.9996
Epoch 4/5
140/140 [==============================] - 83s 594ms/step - loss: 1.0999e-04 - accuracy: 1.0000
Epoch 5/5
140/140 [==============================] - 85s 608ms/step - loss: 5.7851e-05 - accuracy: 1.0000


In [29]:
ls

drive/  sample_data/


In [30]:
cd /content/drive/MyDrive/IBM

/content/drive/MyDrive/IBM


In [31]:
# saving model
model.save('spamdetector.h5')

testing model

In [32]:
from tensorflow.keras.models import load_model

In [33]:
model=load_model('/content/drive/MyDrive/IBM/spamdetector.h5')

In [43]:
def preprocess(text):
  text=re.sub('[^a-zA-Z]',' ',text)
  text=text.lower()
  text=text.split()
  text=[pe.stem(word) for word in text if not word in set(stopwords.words('english'))]
  text=' '.join(text)
  return text

In [46]:
message='Thanks for your subscription to Ringtone UK your mobile will be charged �5/month Please confirm by replying YES or NO. If you reply NO you will not be charged'

In [47]:
message=preprocess(message)

In [48]:
message

'thank subscript rington uk mobil charg month pleas confirm repli ye repli charg'

In [50]:
pred=model.predict(cv.transform([message]))

1/1 [==============================] - 0s 479ms/step


In [52]:
pred

array([[1.]], dtype=float32)

In [53]:
if(pred>0.5):
  print('spam message!!!')
else:
  print('Not Spam')

spam message!!!
